In [5]:
import requests
from bs4 import BeautifulSoup
import time

# 武蔵野大学のトップページURL
base_url = 'https://www.musashino-u.ac.jp/'
domain = 'musashino-u.ac.jp'

# 訪問済みURLと未訪問URLの管理
visited_urls = set()
urls_to_visit = [base_url]

# URL:タイトルの辞書
url_title_dict = {}

# 訪問カウンター（10で終了させるため）
visit_count = 0
max_visits = 10

# サイトマップ抽出
while urls_to_visit and visit_count < max_visits:
    # 次に訪問するURLを取得
    current_url = urls_to_visit.pop(0)
    
    # すでに訪問済みならスキップ
    if current_url in visited_urls:
        continue
    
    # 訪問済みに追加
    visited_urls.add(current_url)
    visit_count += 1
    
    print(f"訪問中: {current_url} ({visit_count}/{max_visits})")
    
    # サーバー負荷軽減のため待機
    time.sleep(0.5)
    
    try:
        # ページを取得
        response = requests.get(current_url, timeout=10)
        
        response.encoding = response.apparent_encoding
        
        # 成功した場合のみ処理
        if response.status_code == 200:
            # HTMLをパース
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # タイトルを抽出
            title_tag = soup.find('title')
            title = title_tag.text if title_tag else "タイトルなし"
            
            # 辞書に追加
            url_title_dict[current_url] = title
            
            # すべてのリンクを取得
            links = soup.find_all('a', href=True)
            
            for link in links:
                href = link['href']
                
                # 相対URLを絶対URLに変換
                if href.startswith('/'):
                    href = base_url + href.lstrip('/')
                elif not (href.startswith('http://') or href.startswith('https://')):
                    href = base_url + href
                
                # 同一ドメインのURLのみを追加
                if domain in href and href not in visited_urls and href not in urls_to_visit:
                    # PDFや画像などのファイルは除外
                    if not href.endswith(('.pdf', '.jpg', '.jpeg', '.png', '.gif', '.zip')):
                        urls_to_visit.append(href)
    
    except Exception as e:
        print(f"エラー: {current_url} - {e}")

print(f"\n合計 {visit_count} ページを訪問しました。")
print("結果:")
for url, title in url_title_dict.items():
    print(f"{url} - {title}")

訪問中: https://www.musashino-u.ac.jp/ (1/10)
訪問中: https://www.musashino-u.ac.jp/#main (2/10)
訪問中: https://ef.musashino-u.ac.jp/donation/ (3/10)
訪問中: https://www.musashino-u.ac.jp/access.html (4/10)
訪問中: https://www.musashino-u.ac.jp/admission/request.html (5/10)
訪問中: https://www.musashino-u.ac.jp/contact.html (6/10)
訪問中: https://www.musashino-u.ac.jp/prospective-students.html (7/10)
訪問中: https://www.musashino-u.ac.jp/students.html (8/10)
訪問中: https://www.musashino-u.ac.jp/alumni.html (9/10)
訪問中: https://www.musashino-u.ac.jp/parents.html (10/10)

合計 10 ページを訪問しました。
結果:
https://www.musashino-u.ac.jp/ - 武蔵野大学
https://www.musashino-u.ac.jp/#main - 武蔵野大学
https://ef.musashino-u.ac.jp/donation/ - ご寄付のお願い | 学校法人武蔵野大学
https://www.musashino-u.ac.jp/access.html - 交通アクセス | 武蔵野大学
https://www.musashino-u.ac.jp/admission/request.html - 資料請求 | 入試情報 | 武蔵野大学
https://www.musashino-u.ac.jp/contact.html - お問い合わせ | 武蔵野大学
https://www.musashino-u.ac.jp/prospective-students.html - 武蔵野大学で学びたい方 | 武蔵野大学
https://www